# Introduction to STEM-EDX Analysis Using Hyperspy

This is a brief introduction to using the STEM-EDX analysis tools in Hyperspy. We will cover extracting elemental images and quantifying the composition of a specific region of an image.

The example dataset here is a STEM-EDX spectrum image of a single PtNi nanoparticle.

# 1. Loading and Viewing Data

Import Hyperspy.

In [ ]:
import hyperspy.api as hs
%matplotlib nbagg

Load both the ADF image and the spectrum image.

In [ ]:
adf_filename = "HAADF Image.dm4"
si_filename = "EDS Spectrum Image.dm4"
adf = hs.load(adf_filename)
si = hs.load(si_filename)

Initial plotting of the spectrum image data show a summed intensity for the navigator and the spectrum from an individual pixel as the signal.

In [ ]:
si.plot()

We can view the summed spectrum by switching the navigation and signal axes or by simply summing over the navigation axes.

In [ ]:
si.

# 2. Extracting Elemental Maps

In order to extract elemental maps we need to first check / set a number of parameters. Let's take a look at what's already contained in our Hyperspy signal metadata.

In [ ]:
si.

In order to map any elements we first need to set those elements within the data.

In [ ]:
si.set_elements()

Checking the metadata again we can see that those elements have been added.

In [ ]:
si.metadata

Additionally, you have to assign the specific X-ray peaks for each element. If you've already set the elements you can do this using the add_lines() function.

In [ ]:
si.add_lines()

The add_lines() function with no arguments will add only one peak per element (the highest energy alpha peak within the energy range or below an overvoltage of 2). We can add any others manually.

In [ ]:
si.add_lines()

In [ ]:
si.metadata

To visualize the lines you have just added you can plot the sum spectrum with the added lines.

In [ ]:
si.

If unknown peaks appear in your spectrum you can identify them based on energy using the get_xray_lines_near_energy() function.

In [ ]:
hs.eds.get_xray_lines_near_energy()

In [ ]:
si.add_lines()

In [ ]:
si.sum().plot(True)

One additional problem (particularly for the ARM 200 here) is that the offset of the energy axis is slightly off. If you didn't correct for this at the microscope you can do it now.

In [ ]:
si.axes_manager[-1].offset = 

In [ ]:
si.sum().plot(True)

Once you have set all of the elements you would like to map and made sure that they're in the right place on the energy axis you can plot the elemental maps.

In [ ]:
si.get_lines_intensity(plot_result=True)

In [ ]:
si.sum().plot()

# 3. Quantification

While we could quantify the spectrum of the whole image it would seem more logical here to just quantify the spectrum from the nanoparticle in this case. We can do this by getting a mask that just encompasses the nanoparticle and applying it to the spectrum image to get a spectrum just from that region.

In [ ]:
mask = adf > 2.3*10**8
mask.plot()

In [ ]:
s_particle = 
s_particle.plot(True)

For quantification we need to extract the intensities under the peaks. This means we need to subtract the background signal. There are two main methods for doing this:

1. Define background windows set the background signal above and below the peak.

2. Model and fit the spectrum including the background.

1. Using background windows.

In [ ]:
bw = s_particle.

In [ ]:
s_particle.plot(background_windows=bw)

In [ ]:
s_particle_int_bw = s_particle.

2. Fitting a model

In [ ]:
m = s.create_model()

In [ ]:
m.fit()

In [ ]:
m.fit_background()

In [ ]:
m.calibrate_energy_axis(calibrate='resolution')

In [ ]:
m.plot()

In [ ]:
s_particle_int_fit = m.get_lines_intensity()

Now we have the intensity under the peaks we can use one of the quantification methods discussed previously. For Cliff-Lorimer, k-factors for both ePSIC microscopes at the highest accelerating voltages can be found at https://confluence.diamond.ac.uk/display/EPSICWEB/Calibrations.

In [ ]:
kfactors = [1.1,2.9]

In [ ]:
s_particle.set_elements(['Ni','Pt'])
s_particle.set_lines(['Ni_Ka','Pt_La'])

In [ ]:
composition_bw = s_particle.quantification([s_particle_int_bw[2],s_particle_int_bw[4]],method='CL',factors=kfactors)
composition_fit = s_particle.quantification([s_particle_int_fit[3],s_particle_int_fit[5]],method='CL',factors=kfactors)

In [ ]:
print('             |-----------------------------|')
print('             |     Atomic compositions     |')
print('             |-----------------------------|')

print(' \t     |    Bg sub   |     Fitted    |')
print('|------------|-------------|---------------|')
print('| Ni (at. %) |     {:.3}]    |      {:.3}]     |'.format(str(composition_bw[0].data), str(composition_fit[0].data)))
print('| Pt (at. %) |     {:.3}]    |      {:.3}]     |'.format(str(composition_bw[1].data), str(composition_fit[1].data)))
print('|------------|-------------|---------------|')


In [ ]:
s_particle.metadata

In [ ]:
s_particle.set_microscope_parameters(beam_current=0.1)
s_particle.set_microscope_parameters(live_time=0.005)

In [ ]:
zfactors = [1,2]

In [ ]:
composition_zfactors = s_particle.quantification([s_particle_int_fit[3],s_particle_int_fit[5]],method='zeta',factors=zfactors)

In [ ]:
composition_fit